# Methods to Sonify a Pulse

## Astronify

In [1]:
import numpy as np
from astropy.table import Table
from astronify.series import SoniSeries

down_rate  = 10

data       = np.load('Data/Burst.npy')
data       = np.mean(data, axis=1)
data       = (data - data.min()) / (data.max() - data.min())
data       = np.mean(data[:len(data) // down_rate * down_rate].reshape(-1, down_rate), axis=1)

data_table = Table({
    'time': np.arange(len(data)),
    'flux': data
})

data_soni = SoniSeries(data_table)
data_soni.note_spacing = 0.01
data_soni.sonify()
data_soni.write('Audio.wav')

## Profile To Wave

In [2]:
import numpy as np
from scipy import interpolate
import matplotlib.pyplot as plt

wave_length = 10 # second
sample_rate = 48000
repeat_num  = 10

data        = np.load('Data/Burst.npy')
data        = np.mean(data, axis=1)
data        = np.tile(data, repeat_num)

time        = np.arange(len(data)) / (len(data) - 1) * wave_length
f           = interpolate.interp1d(time, data, kind='linear')
wave_time   = np.arange(0, wave_length, 1/sample_rate)
wave_raw    = f(wave_time)
# wave_raw    = ((wave_raw - np.min(wave_raw)) / (wave_raw.max() - wave_raw.min()) * 6e4 - 3e4).astype(np.int16)
wave_raw    = ((wave_raw - np.min(wave_raw)) / (wave_raw.max() - wave_raw.min()) * 6e4).astype(np.int16)

### Convolution

In [4]:
import wave
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from scipy import integrate, signal

def read_wave(file):
    with wave.open(file, 'rb') as f:
        nchannels, sampwidth, framerate, nframes = f.getparams()[:4]
        str_data                                 = f.readframes(nframes)
    wave_data       = np.frombuffer(str_data, dtype=np.short)
    wave_data.shape = -1, nchannels
    return wave_data.T[0]

# 读取音频文件
# pulse      = read_wave('Audio.wav')
pulse      = wave_raw
sound      = read_wave('Instruments/vio.wav')

# 归一化乐器声音
sounds     = stats.binned_statistic(
    x      = np.arange(0, len(sound)), 
    values = sound, 
    bins   = len(sound) * 44100 / 48000
)[0]

L          = integrate.simps(sounds)
sound_norm = sounds / L

# 卷积
wave_raw   = (signal.convolve(pulse, sound_norm, mode='same') / 10).astype(np.int16)

## Intensity To Loudness

In [6]:
import numpy as np
from scipy.interpolate import interp1d

def sin_fu(x, a, b):
    return a * np.sin(1000 * x) + b

sampling_rate = 48000
wave_length   = 2
int_number    = sampling_rate * wave_length

b = np.load('Data/Profile.npy')
b = np.log10((b - np.min(b)) / (b.max() - b.min()) + 1)
a = np.linspace(0, 2*np.pi, len(b))

c = interp1d(a, b)
d = np.linspace(a.min(), a.max(), int_number)
e = c(d)
g = sin_fu(d, e, e)
wave_raw = (g * 50000).astype(np.int16)

In [7]:
# import soundfile as sf
# sf.write('Audio.wav', wave_raw, 48000, subtype='PCM_24')

import wave
with wave.open('Audio.wav', 'wb') as f:
    params = (1, 2, 48000, 1, 'NONE', 'not compressed')
    f.setparams(params)
    f.writeframesraw(wave_raw)

## Mel Spectrum

In [8]:
import numpy as np
import librosa, copy
import soundfile as sf
from scipy import signal

def del_burst(data, exposure_cut=25):
    h, w            = data.shape
    data           /= np.mean(data, axis=0)
    flatten_data    = np.sort(data.flatten())
    vmin, vmax      = flatten_data[int(h*w/exposure_cut)], flatten_data[int(h*w/exposure_cut*(exposure_cut-1))]
    data[data<vmin] = vmin
    data[data>vmax] = vmax
    data            = (data - data.min()) / (data.max() - data.min())
    return data

###### 消色散后 ######
# data            = np.load('Data/Burst.npy')
# data            = data.astype(np.float64)
# data            = del_burst(data)
# data            = data[1024: 1024+2048]
# data            = np.mean(data.reshape(64, 32, 512, 8), axis=(1, 3))

###### 消色散前 ######
data            = np.load('Data/RawBurst.npy')
data            = np.mean(data.reshape(128, 22, 512, 8), axis=(1, 3))

In [9]:
def melspectrogram2wav(mel):
    # transpose
    mel    = mel.T
    # de-noramlize
    mel    = (np.clip(mel, 0, 1) * max_db) - max_db + ref_db
    # to amplitude
    mel    = np.power(10.0, mel * 0.05)
    m      = _mel_to_linear_matrix(sr, n_fft, n_mels)
    mag    = np.dot(m, mel)
    # wav reconstruction
    wav    = griffin_lim(mag)
    # de-preemphasis
    wav    = signal.lfilter([1], [1, -preemphasis], wav)
    # trim
    wav, _ = librosa.effects.trim(wav)
    return wav.astype(np.float32)

def spectrogram2wav(mag):
    # transpose
    mag    = mag.T
    # de-noramlize
    mag    = (np.clip(mag, 0, 1) * max_db) - max_db + ref_db
    # to amplitude
    mag    = np.power(10.0, mag * 0.05)
    # wav reconstruction
    wav    = griffin_lim(mag)
    # de-preemphasis
    wav    = signal.lfilter([1], [1, -preemphasis], wav)
    # trim
    wav, _ = librosa.effects.trim(wav)
    return wav.astype(np.float32)

def _mel_to_linear_matrix(sr, n_fft, n_mels):
    m      = librosa.filters.mel(sr=sr, n_fft=n_fft, n_mels=n_mels)
    m_t    = np.transpose(m)
    p      = np.matmul(m, m_t)
    d      = [1.0 / x if np.abs(x) > 1.0e-8 else x for x in np.sum(p, axis=0)]
    return np.matmul(m_t, np.diag(d))

def griffin_lim(spectrogram):
    X_best     = copy.deepcopy(spectrogram)
    for i in range(n_iter):
        X_t    = invert_spectrogram(X_best)
        est    = librosa.stft(X_t, n_fft=n_fft, hop_length=hop_length, win_length=win_length)
        phase  = est / np.maximum(1e-8, np.abs(est))
        X_best = spectrogram * phase
    X_t        = invert_spectrogram(X_best)
    y          = np.real(X_t)
    return y

def invert_spectrogram(spectrogram):
    return librosa.istft(spectrogram, n_fft=n_fft, hop_length=hop_length, win_length=win_length, window='hann')

In [10]:
### Params
sr           = 48000  # Sample rate.
n_iter       = 32   # Number of inversion iterations
preemphasis  = 0.97 # or None
max_db       = 100
ref_db       = 20
top_db       = 15
n_mels       = 512  # Number of Mel banks to generate
n_fft        = 8190

### Mel To Wave

In [11]:
### Mel to Wav
frame_length = 0.04   # seconds
win_length   = int(sr * frame_length) # samples.
hop_length   = win_length // 4        # samples.

a            = melspectrogram2wav(data)
sf.write('Audio.wav', a, sr, subtype='PCM_24')

### Mag To Wave

In [14]:
### Mag to Wav
hop_length   = 19
win_length   = hop_length * 4

data         = np.load('Data/RawBurst.npy')
asd          = spectrogram2wav(data)
sf.write('Audio.wav', asd, sr, subtype='PCM_24')